# Lab 10: Data wrangling with tables

Welcome to lab 10!  This lab is based on a homework problem from the 5th week of the course.  You'll combine together several datasets to show that places near water have more moderate (less variable) climates.  There is less focus on inferential questions, and more on "wrangling" the data, using important tools like `join` and `apply`.  If you don't have prior programming experience, don't expect to finish this lab in an hour.

We've also included a companion notebook we released to help students who were struggling with this exercise.  If you're interested, you can find that [here](longlab10_guide.ipynb).

As usual, **run the cell below** to prepare the lab and the automatic tests.

In [26]:
# Run this cell, but please don't change it.

# These lines import the NumPy and datascience modules.
import numpy as np
# This way of importing the datascience module lets you write "Table" instead
# of "datascience.Table".  The "*" means "import everything in the module."
from datascience import *

# These lines set up visualizations.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.assignment import load_assignment 
lab10 = load_assignment('longlab10.ok')

# 1. Climate and water
To get more practice analyzing data with tables, you'll work with a dataset about climate.  Theory says that land masses near large bodies of water have more moderate temperatures, since the water cools the land when it's hot and warms it when it's cold.  You'll be investigating whether that means places near coastlines have smaller daily temperature fluctuations than places far away from coastlines.

The US National Oceanic and Atmospheric Administration (NOAA) operates thousands of weather stations (mostly in the US) that collect information about local climate.  Among other things, each station records the highest and lowest observed temperature each day.  These data, called "Quality Controlled Local Climatological Data," are publicly available [here](http://www.ncdc.noaa.gov/orders/qclcd/) and described [here](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd).

The dataset consists of 3 tables:
* The main dataset is a sample of all the daily readings from 2015.  (There were half a million total readings in the original dataset, but that takes some time to process, so we've sampled it for you.)  These are in a table called `daily_temp.csv`.  Each row corresponds to one day at one station, and includes the maximum and minimum temperature (in degrees Fahrenheit) for that day, the day (in MonthDay format), and an identifier, called a WBAN (Weather Bureau Army Navy) ID, for the station that made the reading.  When you read this into a Table, call it `daily_temp`.
* A separate table, called `stations.csv`, contains information about each station.  In particular, it gives the global coordinates (latitude and longitude) of each station, identified by WBAN ID.  Call this Table `stations`.  **Note:** `stations.csv` actually uses the pipe character ("`|`") to separate values, rather than commas.  To read it, use `Table.read_table("stations.csv", sep="|")` .
* A third table, called `coast_distance.csv`, tells us roughly how far away every place on Earth (to a precision of half a degree latitude/longitude) is from a coastline.  The data come from [a NASA project](http://oceancolor.gsfc.nasa.gov/DOCS/DistFromCoast/).  Call this table `coast_distance`.

You will notice that some values are `nan`, which stands for *not a number*. These are values that are missing from the table for some reason. We will not use the columns that have missing values.

**Question 1.1.** Use the next cell to load the tables.

In [27]:
# Load the 3 tables described above.  Read the note about the special
# format of stations.csv.
daily_temp = ...
stations = ...
coast_distance = ...
daily_temp.show(3)
stations.show(3)
coast_distance.show(3)

In [28]:
_ = lab10.grade('q11')

**Question 1.2.** Choose an appropriate visualization to display the distribution of maximum temperatures across all the daily readings *and* the distribution of minimum temperatures across all the daily readings.  Write code that displays both in the same graph.

*Hint:* The visualization methods you've seen allow you to display two columns in one chart by passing a list of labels as the data argument. For example, if `t` is a table and `"a"` and `"b"` are two of its columns, then `t.hist(["a", "b"])` makes histograms of both columns in one graph.  Similarly, if `t` is a table, `"c"` is a column of categories, and `"d"` and `"e"` are columns of counts, then `t.barh("c", ["d", "e"])` makes bar charts of the counts of `"c"` in `"d"` and `"e"`.

In [29]:
# Display the distributions of maximum and minimum temperatures.
# Use just one graph.
...

**Question 1.3.** The temperature spread for a day at a station is the difference between the maximum temperature and minimum temperature at that station for that day.  Make a visualization to display the distribution of temperature spreads for each day.

*Hint:* First construct a new column called `spread`, then visualize it.

In [30]:
# Display the distribution of daily temperature spreads.
...

Below is another cell to help you visualize the data in the `stations` table.  For each station, it plots a single point, which is that station's latitude/longitude coordinates.  Just run it.  What does it look like?  Can you tell where most of the NOAA stations are located in the world?

In [31]:
# Run this cell and try to interpret the scatter diagram it generates.
stations.scatter("Longitude", "Latitude")

Now we're going to see how temperature spread relates to distance from the coast.  `daily_temp` tells us which station each reading came from, but not the distance of that station from a coastline.   `stations` tells us where each station is in the world, but not how far it is from a coastline.  `coast_distance` tells us how far each place in the world is from a coastline.  So this analysis will involve all 3 tables, and it will involve `join`ing them together.  We'll guide you through the steps.

First, we want to build a table called `stations_with_distances` that has the WBAN and distance-to-coast for each station in the `stations` table.  We'll do that over the course of the next few questions.

**Question 1.4.** Look at the first row of the `stations` table, which describes the station at Dexter B. Florence Memorial Field Airport, in the US state of Arkansas.  If you [look it up on a map](http://www.mapdevelopers.com/distance_finder.php?&points=%5B%5B29.80251790576448%2C-93.22998046875%5D%2C%5B34.09865686549439%2C-93.06673049926758%5D%5D), you'll see that this station is about 480 kilometers from the nearest coastline to the South in Louisiana.

Find the (approximate) distance-to-coast for this station, looking at the `coast_distance` table manually.  (Instead of looking through the whole table, you might want to use Python code to narrow your search.)  What is that distance?  *More importantly,* how did you find it?

*Write your answer here, replacing this text.*

In [32]:
# Here's a cell for working on Question 1.4, but only the answer above will be scored.
coast_distance.where(np.logical_and(coast_distance.column('latitude') == 34, coast_distance.column('longitude') == -93, ))

We're going to use `join` to get Python to perform that process for every station.  Here are two observations that pose obstacles to using `join`:
1. The latitudes and longitudes in `coast_distance` are only given to half a degree of precision.  For example, that means there is an entry for (-20, 45.5) (that is, -20 degrees latitude and 45.5 degrees longitude), but not for (-20, 45.6) or (-19.9, 45.5).  By contrast, the latitudes and longitudes in `stations` are given to a precision of ten-thousandths of a degree.  For example, Dexter B. Florence Memorial Field Airport is at (34.0997, -93.0658).
2. We have to match each station in `stations` to a row in `coast_distance` that matches both its latitude *and* its longitude.

**Question 1.5.** Why is the first observation an obstacle for `join`ing `stations` with `coast_distance`?

*Write your answer here, replacing this text.*

**Question 1.6.** Why is the second observation an obstacle for `join`ing `stations` with `coast_distance`?

*Write your answer here, replacing this text.*

Here is how we'll fix these problems:
1. We'll round off the coordinates in `stations` to the nearest half-degree.  That way, each station's rounded-off location will exactly match a location in `coast_distance`.  For example, we'll round Dexter B. Florence Memorial Field Airport's coordinates to (34, -93), which exactly matches the coordinates of row 80094 in `coast_distance`.  Hopefully that's the row you matched it to manually in question 1.4!
2. Then we'll make a single column called `"lat,long"` in both `stations` and `coast_distance` that combines each row's latitude and longitude into a single string.  We'll write functions to do these steps for one entry in our tables, and then we'll use `apply` to fix all the data at once.

(*Note:* You might worry that rounding the stations' coordinates will change their distance-to-coast.  They will be off by a little bit, but that's okay.  For an extra challenge, try to figure out how much error we might be introducing.)

**Question 1.7.** The following function is useful for one of the two steps.  Its argument is a single number, and it returns a number.  Describe what it does. If you're not sure, try calling it on various numbers such as 3, 3.1415, and 0.31415.

*Hint:* Remember that `round` rounds a number to the nearest integer.

In [33]:
def mysterious_function(num):
    return round(2 * num) / 2

*Describe what `mysterious_function` does, replacing this text.*

**Question 1.8.** Write a function called `append_numbers` that takes two numbers as its arguments and returns the string `"<first number>,<second number>"`.  For example, the value of `append_numbers(1.5, -2.0)` should be the string `"1.5,-2.0"`.  (We're just using double-quotes to delimit the string; the return value of `append_numbers` shouldn't include any actual quote characters.  Also, note the comma in between the numbers.)

*Hint 1:* Remember the `str` function, which converts a number to a string that looks like that number.

*Hint 2:* Adding two or more strings together with `+` appends them together.  For example, `"port" + "able"` is `"portable"`.

In [34]:
def append_numbers(num1, num2):
    ...

In [35]:
_ = lab10.grade('q18')

**Question 1.9.** Create two new columns from `stations` called `Rounded Latitude` and `Rounded Longitude`, the result of rounding each latitude and longitude in `stations`, respectively.  Extend `stations` with these columns, calling the resulting table `stations_rounded`.

In [36]:
stations_rounded = stations.with_columns([
        "Rounded Latitude", 
        ...
        "Rounded Longitude", 
        ...
    ])
stations_rounded

In [37]:
_ = lab10.grade('q19')

**Question 1.10.** Create a new column from `stations_rounded` called `lat,long` that is the result of appending the rounded latitudes and longitudes with `append_numbers`.  Extend `stations_rounded` with this column.  Do the same for the latitudes and longitudes in `coast_distance`.

*Hint:* `apply` can combine two or more columns by saying, for example, `my_table.apply(two_arg_function, ["a_column", "another_column"])`.  That will return a single array, which is the result of applying `two_arg_function` to each pair of elements in the columns `"a_column"` and `"another_column"`.

*Hint 2:* Be careful about capitalization of labels. The labels `"Latitude"` and `"latitude"` both appear in different tables.

In [38]:
# Make this the column you want to add to stations_rounded (that is,
# an array of "<latitude>,<longitude>" strings.  The line of code after
# this one will add it to stations_rounded for you.
stations_rounded_lat_long = ...
stations_rounded.append_column("lat,long", stations_rounded_lat_long) # Adds a column to stations_rounded
# Make this the column you want to add to coast_distance (that is,
# an array of "<latitude>,<longitude>" strings.  The line of code
# after this one will add it to coast_distance for you.
coast_distance_lat_long = ...
coast_distance.append_column("lat,long", coast_distance_lat_long)     # Adds a column to coast_distance

In [39]:
_ = lab10.grade('q110')

**Question 1.11.** Now we're ready to join!  Create a new table called `stations_with_distances` that extends `stations_rounded` with a column of distances to the nearest coastline.  That column should be called `"distance to coast (km)"`.  Use `join`. The result will take a minute to compute.

In [40]:
stations_with_distances = ...
stations_with_distances

In [41]:
_ = lab10.grade('q111')

**Question 1.12.** Now we're going to join again to get the station information for each daily reading.  This time it's much easier!  Create a new table called `daily_temp_with_distances` that extends `daily_temp` with a column of distances to the nearest coastline, called `"distance to coast (km)"`.  Use `join` again.

*Hint:* What column of identifiers do `daily_temp` and `stations_with_distances` share?

In [42]:
daily_temp_with_distances = ...
daily_temp_with_distances

In [43]:
_ = lab10.grade('q112')

**Question 1.13.** Whew!  Now let's do the actual analysis we set out to do.  Compute the average temperature spread for stations that are more than 50 kilometers from a coastline and for those closer to a coastline.  Do this by computing a table called `spread_for_coastal_and_noncoastal` that looks like this **(except that these aren't the right numbers)**:

    >50km from coast | spread (°F) average
    False            | 0.1234
    True             | 5.6789

*Hint:* First, for each daily reading, compute two new columns: (1) the temperature spread, and (2) a boolean value that's True if that reading comes from a station that's more than 50km from the coast.  Elementwise arithmetic and comparison operators will be useful.  Then use `group` to produce the table we've asked for.

In [44]:
# Compute the table spread_for_coastal_and_noncoastal as described above.
# Our solution uses a single statement spread across a few lines, but
# feel free to compute the table however you like.
spread_for_coastal_and_noncoastal = Table().with_columns([
        "spread (°F)", 
        ...
        ">50km from coast", 
        ...
    ]).group(">50km from coast", np.average)
spread_for_coastal_and_noncoastal

In [45]:
_ = lab10.grade('q113')

If you're curious, you can run the following cell to see a scatter plot of temperature spreads versus distance to coast.  We've grouped the data by station to make the graph clearer.

In [46]:
# Just run this cell to see a plot of the temperature spreads.
daily_temp_with_distances.with_column("spread", daily_temp_with_distances.column("Tmax") - daily_temp_with_distances.column("Tmin")) \
                         .group("WBAN", np.average) \
                         .select(["distance to coast (km) average", "spread average"]) \
                         .relabeled("distance to coast (km) average", "distance to coast (km)") \
                         .relabeled("spread average", "yearly average temperature spread (°F)") \
                         .scatter("distance to coast (km)")